In [1]:
import os
# change directory in order to be able to import python scripts from the common package
if os.getcwd().endswith('coinrun'): os.chdir(os.path.expanduser(".."))

In [2]:
from env import CoinRunReplayBuffer, VanillaEnv
import numpy as np
import matplotlib.pyplot as plt
import os
import random
import crafter
from policy import CoinRunActor
import torch
import cv2

In [25]:
folder_name: str = './coinrun/experiments/231028-185708/'

def play_episode(model: CoinRunActor, env)->bool:
    """
    returns true of the episode is concidered solved
    """

    model.eval()
    done = False
    episode_return = 0
    obs = env.reset()
    frames = [obs]
    while not done:       
        action_logits = model.forward(torch.FloatTensor(obs).unsqueeze(0), contrastive=False)
        action = torch.argmax(action_logits)
        obs, rewards, done, info = env.step(action.item())

        
        episode_return += rewards
        frames.append(obs)
    return frames, episode_return



models_names = [n for n in os.listdir(folder_name) if n.endswith('.pth')]
for models_name in models_names:
    model = CoinRunActor()

    ckp = torch.load(folder_name + models_name, map_location=torch.device('cpu'))
    model.load_state_dict(ckp['state_dict'])

    episode_return = -5
    while episode_return < -1:
        frames, episode_return = np.array(play_episode(model, VanillaEnv(start_level=110, num_levels=1)))
        print(episode_return)

    fps = 8
    out = cv2.VideoWriter('output.mp4', cv2.VideoWriter_fourcc(*'DIVX'), fps, (64,64))

    for i in range(len(frames)):
        rgb_img = cv2.cvtColor(np.moveaxis(np.array(frames[i]*255, dtype=np.uint8), 0, -1), cv2.COLOR_RGB2BGR)
        # rgb_img = np.moveaxis(np.array(frames[i]*255, dtype=np.int8), 0, -1)
        out.write(rgb_img)
    out.release()

0.2062745098039419


C:\Users\manus\AppData\Local\Temp\ipykernel_29816\765203631.py:34: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  frames, episode_return = np.array(play_episode(model, VanillaEnv(start_level=105, num_levels=1)))
